In [1]:
import pandas as pd

# Load the datasets
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

# Display the first few rows of the datasets
print("Training Data:")
print(train_data.head())

print("\nTest Data:")
print(test_data.head())


Training Data:
                      ID        date     Item Id  \
0  2022-04-12_B09KDTS4DC  2022-04-12  B09KDTS4DC   
1  2022-04-12_B09MR2MLZH  2022-04-12  B09MR2MLZH   
2  2022-04-12_B09KSYL73R  2022-04-12  B09KSYL73R   
3  2022-04-12_B09KT5HMNY  2022-04-12  B09KT5HMNY   
4  2022-04-12_B09KTF8ZDQ  2022-04-12  B09KTF8ZDQ   

                                           Item Name  ad_spend anarix_id  \
0  NapQueen Elizabeth 8" Gel Memory Foam Mattress...       NaN  NAPQUEEN   
1  NapQueen 12 Inch Bamboo Charcoal Queen Size Me...       NaN  NAPQUEEN   
2     NapQueen Elsa 8" Innerspring Mattress, Twin XL       NaN  NAPQUEEN   
3        NapQueen Elsa 6" Innerspring Mattress, Twin       NaN  NAPQUEEN   
4     NapQueen Elsa 6" Innerspring Mattress, Twin XL       NaN  NAPQUEEN   

   units  unit_price  
0    0.0         0.0  
1    0.0         0.0  
2    0.0         0.0  
3    0.0         0.0  
4    0.0         0.0  

Test Data:
                      ID        date     Item Id  \
0  2024-07-01

# Exploratory Data Analysis

In [2]:
# Check for null values in training data
print("Null values in training data:")
print(train_data.isnull().sum())

# Check for null values in test data
print("\nNull values in test data:")
print(test_data.isnull().sum())


Null values in training data:
ID                0
date              0
Item Id           2
Item Name      1832
ad_spend      24187
anarix_id         0
units         17898
unit_price        0
dtype: int64

Null values in test data:
ID               0
date             0
Item Id          0
Item Name      344
ad_spend      1451
anarix_id        0
unit_price       0
dtype: int64


In [3]:
# Summary statistics of training data
print("Summary statistics of training data:")
print(train_data.describe())

# Summary statistics of test data
print("\nSummary statistics of test data:")
print(test_data.describe())


Summary statistics of training data:
           ad_spend         units     unit_price
count  77303.000000  83592.000000  101490.000000
mean     110.771470     10.284381     106.750922
std      529.303777     68.945915     425.704733
min        0.000000   -173.000000   -8232.000000
25%        0.000000      0.000000       0.000000
50%        4.230000      1.000000       0.000000
75%       44.310000      5.000000       0.000000
max    47934.990000   9004.000000   21557.390000

Summary statistics of test data:
           ad_spend   unit_price
count   1382.000000  2833.000000
mean     198.838032    98.725873
std      797.354508   383.585307
min        0.000000 -1988.180000
25%        0.730000     0.000000
50%       39.200000     0.000000
75%      156.012500     0.000000
max    18724.850000  6870.000000


# Feature Engineering

In [4]:
# Handling missing values
# Impute missing ad_spend with median
from sklearn.impute import SimpleImputer
ad_spend_imputer = SimpleImputer(strategy='median')
train_data['ad_spend'] = ad_spend_imputer.fit_transform(train_data[['ad_spend']])
test_data['ad_spend'] = ad_spend_imputer.transform(test_data[['ad_spend']])

# Impute missing units with 0
units_imputer = SimpleImputer(strategy='constant', fill_value=0)
train_data['units'] = units_imputer.fit_transform(train_data[['units']])

# Drop rows with missing 'Item Id' and 'Item Name' in test data
test_data.dropna(subset=['Item Id', 'Item Name'], inplace=True)


In [5]:
# Convert 'date' column to datetime format
test_data['date'] = pd.to_datetime(test_data['date'])
train_data['date'] = pd.to_datetime(train_data['date'])

# Extract additional features
test_data['day_of_week'] = test_data['date'].dt.dayofweek
test_data['month'] = test_data['date'].dt.month
test_data['day_of_year'] = test_data['date'].dt.dayofyear

train_data['day_of_week'] = train_data['date'].dt.dayofweek
train_data['month'] = train_data['date'].dt.month
train_data['day_of_year'] = train_data['date'].dt.dayofyear

In [6]:
# Display the first few rows to check the new features
print(test_data.head())
print(train_data.head())

                      ID       date     Item Id  \
0  2024-07-01_B09KDR64LT 2024-07-01  B09KDR64LT   
1  2024-07-01_B09KDTS4DC 2024-07-01  B09KDTS4DC   
2  2024-07-01_B09KDTHJ6V 2024-07-01  B09KDTHJ6V   
3  2024-07-01_B09KDQ2BWY 2024-07-01  B09KDQ2BWY   
4  2024-07-01_B09KDYY3SB 2024-07-01  B09KDYY3SB   

                                           Item Name  ad_spend anarix_id  \
0  NapQueen Elizabeth 10" Gel Memory Foam Mattres...      4.23  NAPQUEEN   
1  NapQueen Elizabeth 8" Gel Memory Foam Mattress...      4.23  NAPQUEEN   
2  NapQueen Elizabeth 12" Gel Memory Foam Mattres...      4.23  NAPQUEEN   
3  NapQueen Elizabeth 12" Gel Memory Foam Mattres...      4.23  NAPQUEEN   
4  NapQueen Elizabeth 10" Gel Memory Foam Mattres...    101.72  NAPQUEEN   

   unit_price  day_of_week  month  day_of_year  
0         0.0            0      7          183  
1         0.0            0      7          183  
2         0.0            0      7          183  
3         0.0            0      7       

In [7]:
# Fill missing units in test data with 0
test_data['units'] = test_data.get('units', 0)


In [9]:
def create_lag_features(df, lag=7):
    df = df.copy()
    if 'units' in df.columns:
        for i in range(1, lag + 1):
            df[f'lag_{i}'] = df.groupby('Item Id')['units'].shift(i)
    else:
        print("The 'units' column is missing from the dataframe.")
    return df

# Apply the function
train_data = create_lag_features(train_data, lag=7)
test_data = create_lag_features(test_data, lag=7)

# Check the columns after creating lag features
print(train_data.columns)


Index(['ID', 'date', 'Item Id', 'Item Name', 'ad_spend', 'anarix_id', 'units',
       'unit_price', 'day_of_week', 'month', 'day_of_year', 'lag_1', 'lag_2',
       'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7'],
      dtype='object')


# Model Selection

In [10]:
# Use available columns for features
available_features = ['day_of_week', 'month', 'day_of_year', 'ad_spend']
X = train_data[available_features]  # Feature matrix
y = train_data['units']  # Target variable

# Split the data into training and validation sets
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


# Hyperparameter Tuning

In [12]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

# Define the model
model = XGBRegressor(objective='reg:squarederror')

# Define parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 6, 9],
    'subsample': [0.8, 1.0],
}

# Set up GridSearchCV
grid_search = GridSearchCV(model, param_grid, scoring='neg_mean_squared_error', cv=3, n_jobs=-1)

# Fit GridSearchCV
grid_search.fit(X_train, y_train)

# Best parameters and best score
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best MSE: {-grid_search.best_score_}")

# Train model with best parameters
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_val)
mse = mean_squared_error(y_val, y_pred)
print(f'Validation MSE with best parameters: {mse}')



Best parameters: {'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 100, 'subsample': 1.0}
Best MSE: 2737.558994292369
Validation MSE with best parameters: 1341.7745356133707


# Results

In [13]:
# Create the submission file
submission = test_data[['date', 'Item Id']].copy()
submission['TARGET'] = test_data['units']

# Save to CSV
submission.to_csv('submission.csv', index=False)
